In [ ]:
wk0 = [[f, d] for f, d in zip(fips_order, data_usa["cmi_7days_rolling_avg"][0])]
wk0 = pd.DataFrame(wk0, columns=["fips", "class"])
wk0["week"] = 0

wk1 = [[f, d] for f, d in zip(fips_order, data_usa["cmi_7days_rolling_avg"][1])]
wk1 = pd.DataFrame(wk1, columns=["fips", "class"])
wk1["week"] = 1

wk2 = [[f, d] for f, d in zip(fips_order, data_usa["cmi_7days_rolling_avg"][2])]
wk2 = pd.DataFrame(wk2, columns=["fips", "class"])
wk2["week"] = 2

wk = wk0.append(wk1).append(wk2).reset_index()

In [ ]:
plotmap = gpd.GeoDataFrame.from_features(usa_map_data["features"])
plotmap = plotmap.assign(fips=[int(g) for g in plotmap["GEOID"]])

gdf = plotmap.merge(wk, left_on="fips", right_on="fips", how="right")[["geometry", "class", "fips", "week"]]
gdf["centroid_lon"] = gdf["geometry"].centroid.x
gdf["centroid_lat"] = gdf["geometry"].centroid.y

In [ ]:
import json

In [ ]:
import altair as alt
choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values=choro_json["features"])

In [ ]:
date_slider = alt.binding_range(min=0, max=2, step=1)
slider_selection = alt.selection_single(
    bind=date_slider,
    fields=["properties.week"],
    name="Week #:",
    init={ "properties.week": 1 },
    # on="none"
)

base = alt.Chart(plotmap).mark_geoshape(
    stroke="black",
    fill="#FFFFFF00",
    strokeWidth=1
).encode().properties(width=800, height=600).project(type="albersUsa")


choro = alt.Chart(choro_data).mark_geoshape(
    # fill="lightgray",
    stroke="black"
).encode(
    color=alt.Color("properties.class:O", scale=alt.Scale(scheme="category10")),
    tooltip=["properties.fips:O"]
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)
chart = base + choro
chart 

# chart.save("chart.html")

In [ ]:
choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values=choro_json["features"])

In [ ]:
date_slider = alt.binding_range(min=0, max=2, step=1)
slider_selection = alt.selection_single(
    bind=date_slider,
    fields=["week"],
    name="Week #:",
    init={ "week": 1 },
    # on="none"
)

alt.data_transformers.enable("json")

base = alt.Chart(plotmap).mark_geoshape(
    stroke="black",
    fill="#FFFFFF00",
    strokeWidth=1
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
).encode(
    color=alt.Color("properties.class:O", scale=alt.Scale(scheme="category10"))
).transform_lookup(
    lookup="fips",
    from_=alt.LookupData(wk, "fips", ["class"])
).properties(width=800, height=600).project(type="albersUsa")

base